In [ ]:
%matplotlib inline
import numpy as np
import keras.backend as K
from keras import Sequential, Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten
import copy

import Environment as E
import importlib
import matplotlib.pyplot as plt

## Parameters

In [ ]:
# General

n_agents = 1

# Model

time_steps = 3
grid_size_w = 20
grid_size_h = 20
n_actions = 5

n_channels = time_steps + time_steps + 1

# Q-Learning

n_episodes = 10
n_steps = 1000000
epsilon = 0.75
epsilon_decay = 0.99
gamma = 0.9

# Policy Gradient

n_runs = 5

## Model

In [ ]:
data_i = Input(shape=(time_steps, grid_size_w, grid_size_h))
conv_1 = Conv2D(16, (4, 4), activation="relu", data_format="channels_first")(data_i)
conv_2 = Conv2D(32, (2, 2), activation="relu", data_format="channels_first")(conv_1)
pool_1 = MaxPooling2D(data_format="channels_first")(conv_2)
drop_1 = Dropout(0.25)(pool_1)
flat_1 = Flatten()(drop_1)
feed_1 = Dense(64, activation="relu")(flat_1)
drop_2 = Dropout(0.25)(feed_1)
feed_2 = Dense(n_actions, activation="linear")(drop_2)
data_o = feed_2

K.print_tensor(data_o)

model = Model(inputs=data_i, outputs=data_o)

model.compile(optimizer="adam", loss="mse")

model.summary()

## Q-Learning

In [ ]:
def tuple_to_batch(known_data):
    batch = []
    for i,agent_pos in enumerate(known_data[2]):
        batch.append(np.concatenate((known_data[0],known_data[1],known_data[2][i:i+1])))
    batch = np.array(batch)
    return batch

### 

In [ ]:
importlib.reload(E)

for i in range(n_episodes):
    env = E.Environment((20,20))
    d = []
    action_avg = np.zeros((400,5))
    list_avg_rwd = []
    list_steps = []
    for k in range(n_agents):
        env.add_agent(coord=(k,0),capacity=100000)
    
    X = env.export_known_data()
    X = tuple_to_batch(X)
    Q_vals = model.predict(X)
    if i % 10 == 0:
        print("Episode {} of {}".format(i + 1, n_episodes))
    
    r_sum = 0
    
    for step in range(n_steps):
        if np.random.random() < epsilon:
            actions = np.random.randint(0, high=n_actions, size=n_agents).tolist()
        else:
            actions = [np.argmax(q_array) for q_array in Q_vals]
        
        X_reward = env.move_agents(actions) #new known state and rewards
        # after all agents move
        X_new = tuple_to_batch(X_reward[:3])
        rewards = X_reward[3]
        action_avg[step%400, : ] = 0
        action_avg[step%400,actions[0]]= 1
        d.append(rewards[0])
        while(len(d)>400):
            del(d[0])
        if(step% 1000 == 0): 
            mean = sum(d) / 400.0
            epsilon *= epsilon_decay
            print(epsilon)
            print("In Step {} the average reward of 100 is {} ".format(step, mean))
            print("Actions: {}".format(np.sum(action_avg, axis = 0)))
            list_avg_rwd.append(mean)
            list_steps.append(step)
            plt.scatter(list_steps, list_avg_rwd)
            plt.show()
        Q_vals_new = model.predict(X_new)
        
        Q_vals[:, actions] = rewards + gamma * np.max(Q_vals_new, axis=1)
        model.fit(X, Q_vals, epochs=1, verbose=0)
        
        Q_vals = Q_vals_new
        X = X_new